In [2]:
import time
from io import StringIO

import pandas as pd
import requests
import duckdb

In [5]:
# geodaten via https://opendata.swiss/de/dataset/geodaten-zu-den-eidgenoessischen-abstimmungsvorlagen
topojson = requests.get('https://dam-api.bfs.admin.ch/hub/api/dam/assets/22344268/master').json()
# nur gemeinden im kanton bern behalten
topojson['objects']['K4voge_20220501_gf']['geometries'] = [g for g in topojson['objects']['K4voge_20220501_gf']['geometries'] if g['properties']['kantName'] == 'Bern']
be_gemeinde_nummern = dict([(g['properties']['vogeId'], g['properties']['vogeName']) for g in topojson['objects']['K4voge_20220501_gf']['geometries']])

try:
    result_df = pd.read_csv('gemeinde_resultate_listen.csv', encoding='utf-8')
except FileNotFoundError:
    result_df = None
    for g in be_gemeinde_nummern.keys():
        try:
            resp = requests.get(f'https://www.bewas.sites.be.ch/2022/2022-03-27/WAHL_GROSSRAT/reportCsvGemeindeResultat-{g}.csv')
        except requests.ConnectionError as e:
            print('failed to download', g, be_gemeinde_nummern[g], f'https://www.bewas.sites.be.ch/2022/2022-03-27/WAHL_GROSSRAT/reportCsvGemeindeResultat-{g}.csv', e)
            continue
        time.sleep(0.3)
        data = resp.content.decode('windows-1252')
        lines = data.splitlines()
        total_idx = [i for i in range(len(lines)) if lines[i].startswith('Total;')][0]
        stimmberechtigte = int(lines[total_idx+1].split(';')[1])
        wahlzettel = int(lines[total_idx+2].split(';')[1])
        lines = lines[5:total_idx]
        lines.insert(0, 'Listennummer;Partei;Stimmen;Stimmen-Prozent')
        df = pd.read_csv(StringIO('\n'.join(lines)), sep=';', converters={'Stimmen': lambda s: int(s.replace('\'', '')), 'Stimmen-Prozent': lambda s: float(s.replace('%', ''))})
        df['Gemeinde-Nummer'] = g
        df['Gemeinde-Name'] = be_gemeinde_nummern[g]
        df['Stimmberechtigte'] = stimmberechtigte
        df['Wahlzettel'] = wahlzettel
        df['Stimmbeteiligung'] = wahlzettel / stimmberechtigte
        result_df = df if result_df is None else pd.concat([result_df, df])
    result_df.to_csv('gemeinde_resultate_listen.csv', index=False, encoding='utf-8')

In [8]:
duckdb.query('''
    select "Gemeinde-Nummer", "Gemeinde-Name", "Stimmbeteiligung", sum(Stimmen) "Stimmen", round(sum("Stimmen-Prozent"), 2) "Stimmen-Prozent"
    from result_df
    where "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB')
    group by 1, 2, 3
    having sum("Stimmen-Prozent") >= 10
    order by 3 desc
    ''').to_df()

,Gemeinde-Nummer,Gemeinde-Name,Stimmbeteiligung,Stimmen,Stimmen-Prozent
0,715,Rebévelier,0.515152,21.0,10.82
1,716,Petit-Val,0.475460,187.0,10.21
2,866,Gerzensee,0.473456,853.0,10.13
3,307,Meikirch,0.456291,2800.0,14.44
4,352,Bolligen,0.435199,6717.0,14.90
...,...,...,...,...,...
95,442,Romont (BE),0.204969,42.0,11.05
96,441,Renan (BE),0.196477,253.0,14.77
97,435,La Ferrière,0.182464,200.0,22.83
98,449,Sauge,0.172638,249.0,20.56


In [13]:
duckdb.query('''
    select "Gemeinde-Nummer", "Gemeinde-Name", round (100 * "Stimmbeteiligung", 2) "Stimmbeteiligung", sum("Stimmen") "Stimmen",
        sum(case when "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB') then "Stimmen" else 0 end) "Grüne-Stimmen",
        round(sum(case when "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB') then "Stimmen-Prozent" else 0 end), 2) "Grüne-Stimmen-Prozent"
    from result_df
    group by 1, 2, 3
    having sum("Stimmen") > 15000 or sum(case when "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB') then "Stimmen-Prozent" else 0 end) > 12
    order by 4 desc
    ''').to_df().to_csv('grossratswahlen 2022 gruene gemeinden.csv', encoding='utf-8', index=False)

In [91]:
duckdb.query('''
    select *
    from result_df
    where "Gemeinde-Nummer" = 306
    ''').to_df()

,Listennummer,Partei,Stimmen,Stimmen-Prozent,Gemeinde-Nummer,Gemeinde-Name
0,1,Die Mitte / Le Centre,7348,9.025253,306,Lyss
1,2,SVP,13732,16.866463,306,Lyss
2,3,UDC,325,0.399184,306,Lyss
3,4,EVP / PEV,4527,5.560332,306,Lyss
4,5,*jevp / *jpev,1459,1.792031,306,Lyss
5,6,EDU / UDF,1578,1.938194,306,Lyss
6,7,SP Frauen,10077,12.377174,306,Lyss
7,8,SP Männer,5890,7.234450,306,Lyss
8,9,PSR,383,0.470424,306,Lyss
9,10,JUSO / JS,975,1.197553,306,Lyss
